# HW2: Problem 2: Working out Backpropagation

Read Chapter 2 of Michael Nielsen's article/book from top to bottom:

* [http://neuralnetworksanddeeplearning.com/chap2.html](http://neuralnetworksanddeeplearning.com/chap2.html)

He outlines a few exersizes in that article which you must complete. Do the following a, b, c:

a. He invites you to write out a proof of equation BP3

b. He invites you to write out a proof of equation BP4

c. He proposes that you code a fully matrix-based approach to backpropagation over a mini-batch. Implement this with explanation where you change the notation so that instead of having a bias term, you assume that the input variables are augmented with a "column" of "1"s, and that the weights $w_0$.

Your submission should be a single jupyter notebook. Use markdown cells with latex for equations of a jupyter notebook for each proof for "a." and "b.". Make sure you include text that explains your steps. Next for "c" this is an implementation problem. You need to understand and modify the code the Michael Nielsen provided so that instead it is using a matrixed based approach. Again don't keep the biases separate. After reading data in (use the iris data set), create a new column corresponding to $x_0=1$, and as mentioned above and discussed in class (see notes) is that the bias term can then be considered a weight $w_0$. Again use markdown cells around your code and comments to explain your work. Test the code on the iris data set with 4 node input (5 with a constant 1), three hidden nodes, and three output nodes, one for each species/class.

## a. Proof of Michael Nielsons equation BP3
Equation 3:

$\frac{\partial C}{\partial b_j^l}$ = $\delta_j^l$


Then we can apply the chain rule to find the derivative of C with respect to b.


$\frac{\partial C}{\partial b_j^l}$ = $\frac{\partial C}{\partial Z_j^l} . \frac{\partial Z_j^l}{\partial b_j^l}$

If we know that

$\frac{\partial C}{\partial Z_j^l}$ = $\delta_j^l$

then we get:

$\frac{\partial C}{\partial b_j^l}$ =  $\delta_j^l$ . $\frac{\partial Z_j^l}{\partial b_j^l}$


We know that b is a linear transformation of Z:

$Z_j^l = \sum_k w_{jk}^l a_k^{l-1} + b_j^l$

Then we can find the derivative of Z with respect to b, which is simply 1:

$\frac{\partial Z_j^l}{\partial b_j^l}$ = 1

Then we finally arrive at the initial BP3 equation:

$\frac{\partial C}{\partial b_j^l}$ = $\delta_j^l$

## b. Proof of Michael Nielsons equation BP4

Equation 4:


$\frac{\partial C}{\partial w_{jk}^l}$ = $\delta_j^l$ . $a_k^{l-1}$

We can again apply the chain rule:

$\frac{\partial C}{\partial w_{jk}^l}$ = $\frac{\partial C}{\partial Z_j^l} . \frac{\partial Z_j^l}{\partial w_{jk}^l}$

Because z is a linear transformation of w:

$Z_j^l = \sum_k w_{jk}^l a_k^{l-1} + b_j^l$

we can find the derivative of Z with respect to w:

$\frac{\partial Z_j^l}{\partial w_{jk}^l}$ = $a_k^{l-1}$

and as we also know that: 

$\frac{\partial C}{\partial Z_j^l}$ = $\delta_j^l$

And we finally arrive at the initial BP4 equation:

$\frac{\partial C}{\partial w_{jk}^l}$ = $\delta_j^l$ . $a_k^{l-1}$


## c. Using both markdown cells and code cells implement that you code a fully matrix-based approach to backpropagation over a mini-batch. Implement this with explanation where you change the notation so that instead of having a bias term, you assume that the input variables are augmented with a "column" of "1"s, and that the weights $w_0$.

In [1]:

import random
import numpy as np
import pandas as pd

class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k + mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        x_matrix_0 = [x for x, y in mini_batch]
        y_matrix_0 = [y for x, y in mini_batch]
        x_matrix = np.concatenate(x_matrix_0, axis=1)
        y_matrix = np.concatenate(y_matrix_0, axis=1)

        nabla_b, nabla_w = self.backprop(x_matrix, y_matrix)

        self.weights = [w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb
                    for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]  # list to store all the activations, layer by layer
        zs = []  # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + np.kron(b, np.ones([1, y.shape[1]]))
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = np.reshape([np.sum(nb) for nb in delta], [delta.shape[0], 1])
        for _d, _a in zip(delta.transpose(), activations[-2].transpose()):
            _d = np.reshape(_d, [len(_d), 1])
            _a = np.reshape(_a, [len(_a), 1])
            nabla_w[-1] += np.dot(_d, _a.transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = np.reshape([np.sum(nb) for nb in delta], [delta.shape[0], 1])
            for _d, _a in zip(delta.transpose(), activations[-l-1].transpose()):
                _d = np.reshape(_d, [len(_d), 1])
                _a = np.reshape(_a, [len(_a), 1])
                nabla_w[-l] += np.dot(_d, _a.transpose())
        return nabla_b, nabla_w

    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

    def sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))


    def sigmoid_prime(z):
        return sigmoid(z) * (1 - sigmoid(z))

    

In [2]:
#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)




In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import pandas as pd

# Load the iris dataset
iris = load_iris()

# Shuffle the dataset
indices = np.arange(iris.data.shape[0])
np.random.shuffle(indices)

# Split the dataset into a training set and a test set
train_size = int(0.8 * len(indices))  # 80% of data for training
train_indices = indices[:train_size]
test_indices = indices[train_size:]

def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y] 
X =iris.data
y = iris.target
#####
data = pd.DataFrame(X, columns=iris.feature_names)
data.insert(0, "bias padding", 1,)


data['label'] = y

# X = np.insert(X, 0, 1, axis=1)

X = np.insert(X, 0, 1, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def vectorized_result(j,nClasses):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a class
    (0...n) into a corresponding desired output from the neural
    network."""
    e = np.zeros((nClasses, 1))
    e[j] = 1.0
    return e

training_inputs = [np.reshape(x, (5, 1)) for x in X_train]
training_results = [vectorized_result(y,3) for y in y_train]
training_data = list(zip(training_inputs, training_results))
validation_inputs = [np.reshape(x, (5, 1)) for x in X_test]
validation_data = list(zip(validation_inputs, y_test))

model = Network([5, 3, 3])
model.SGD(training_data=training_data,epochs = 30, mini_batch_size = 10, eta = 3, test_data=validation_data)

